<big><b><i>Parsing Script: 2021 brainSPORT rsfMRI TBI Literature Review</i></b></big>


<b>README</b>: This script was written July 14th, 2021 by Daniel Frees, an undergraduate student of UCLA for use by a research team at the Steve Tisch brainSPORT Lab. 

Input directories for the CSV files containing your meta-analysis info. Note that this script is written with the particular categories of our literature in mind, but could be easily edited for other purposes.

First, there is a data cleaning task which will produce a pandas dataframe with all classifications found in the CSV. If there is an error in this step (ie: a certain study did not have information for a given classification), then that will be marked for review.

Next, the pandas dataframe can be passed along to produce a tally of results across all networks, for each individual network, and for other specific purposes (ie: we are using this to investigate chronicity at each severity level). 

<b>DETAILED INSTRUCTIONS:</b>

In [2]:
import pandas as pd
import numpy as np
import csv

pd.set_option("display.max_rows", None, "display.max_columns", None)


<i>Setting up directories to CSV files and cleaning the CSVs:</i>

In [34]:
class Directory():
    def __init__(self):
        self.dmn = "PROVIDE DIRECTORY"
        self.ecn = "PROVIDE DIRECTORY"
        self.limbic = "PROVIDE DIRECTORY"
        self.salience = "PROVIDE DIRECTORY"
        self.dan = "PROVIDE DIRECTORY"
        self.van = "PROVIDE DIRECTORY"
        self.sensorimotor = "PROVIDE DIRECTORY"
        self.visual = "PROVIDE DIRECTORY"

dir = Directory()
#edit your directories for the csv files on your system here

baseDir = "/Users/danielfrees/Desktop/LitReview_CSVs"
dir.dmn = baseDir + "/rsfMRI_TBI_Data - DMN.csv"
dir.ecn = baseDir + "/rsfMRI_TBI_Data - ECN.csv"
dir.limbic = baseDir + "/rsfMRI_TBI_Data - Limbic.csv"
dir.salience = baseDir + "/rsfMRI_TBI_Data - Salience.csv"
dir.dan = baseDir + "/rsfMRI_TBI_Data - DAN.csv"
dir.van = baseDir + "/rsfMRI_TBI_Data - VAN.csv"
dir.visual = baseDir + "/rsfMRI_TBI_Data - Visual.csv"
dir.sensorimotor = baseDir + "/rsfMRI_TBI_Data - Sensorimotor.csv"


dmn = pd.read_csv(dir.dmn)
ecn = pd.read_csv(dir.ecn)
limbic = pd.read_csv(dir.limbic)
salience = pd.read_csv(dir.salience)
dan = pd.read_csv(dir.dan)
van = pd.read_csv(dir.van)
visual = pd.read_csv(dir.visual)
sensorimotor = pd.read_csv(dir.sensorimotor)


data = [dmn, ecn, limbic, salience, dan, van, visual, sensorimotor]

#edit the following based on the order of your imported networks. IMPORTANT for the next cell's function to count correctly
dataOrder = ["dmn", "ecn", "limb", "sn", "dan", "van", "vis", "smn"]

for index in range(len(data)):
    data[index] = data[index].fillna('')
    
#BEFORE PROCESSING
#for i in range(len(data)):
#    display(data[i])

#list of words used for decrease, increase, etc. 
decList = ["DECREASE", 'decrease', 'Decrease', 'Dec', 'dec', 'hypo', 'Hypo', 'HYPO', 'Reduce', 'reduce']
incList = ["INCREASE", 'increase', 'Increase', 'Inc', 'inc', 'hyper', 'Hyper', 'HYPER']
nullList = ["NULL", "NO", "No", "no", 'null', 'Null', 'No diff', 'no diff', 'No Diff', 'No sig', 'no sig', 'No Sig', 'NO SIG']
resultWords = decList + incList + nullList


#cleanData function ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
def cleanData(datalist):
    for i in range(len(datalist)):
        for index, row in datalist[i].iterrows():
            #CLEAN RESULTS SECTION
            #NOTE: First occurence is counted, because that was how we reported our main findings (always came first). Sometimes the other key words will appear describing a finding which 
            #did not survive due to ANCOVA etc. 
            
            #strip all leading and ending whitespace for the columns to be cleaned (important so that I can check for empty cases vs. error cases)
            row['RESULT'].strip()
            row['TBI Class'].strip()
            row['Severity'].strip()
            row['Age'].strip()
            row['Chronicity'].strip()
            row['Control Type'].strip()            
            
            keyPositions = ((row['RESULT'].find(resWord), resWord) for resWord in resultWords)
            keyPositions = [key for key in keyPositions if key[0] != -1]  #don't want to consider keywords which didn't show up (pos -1)
            try:
                leftmost = min(keyPositions, key = lambda t: t[1]) #anonymous func! yay!
                if leftmost[1] in decList:
                    datalist[i].at[index, 'RESULT'] = "dec"
                elif leftmost[1] in incList:
                    datalist[i].at[index, 'RESULT'] = "inc"
                elif leftmost[1] in nullList:
                    datalist[i].at[index, 'RESULT'] = "null"
            except ValueError as e:
                datalist[i].at[i, 'RESULT'] = "//ERROR//"
            
            
            
            #Clean TBI Class Section
            #allows filling down since we wrote this info only once per paper
            if any(civ in row['TBI Class'] for civ in ["Civilian", "civilian", "civ", "Civ", "CIVILIAN"]):
                datalist[i].at[index, 'TBI Class'] = "civilian"
            elif any(sport in row['TBI Class'] for sport in ["sport", "Sport", "Athlete", "athlete", "ATHLETE", "SPORT"]):
                datalist[i].at[index, 'TBI Class'] = "sport"
            elif any(mil in row['TBI Class'] for mil in ["Military", "military", "blast", "Blast", "MILITARY", "BLAST"]):
                datalist[i].at[index, 'TBI Class'] = "military"
            elif any(mix in row['TBI Class'] for mix in ["mix", "Mix", "MIX"]):
                datalist[i].at[index, 'TBI Class'] = "mix Type"    
            elif index-1 >= 0 and datalist[i].at[index, 'TBI Class'] == "":  #fill down
                datalist[i].at[index, 'TBI Class'] = datalist[i].at[index -1, 'TBI Class']
            else: 
                datalist[i].at[index, 'TBI Class'] = "//ERROR//"
            
                
            #Clean Severity Section
            #allows filling down since we wrote this info only once per paper
            mildTerms = ["MILD", "mild", "Mild"]
            modTerms = ["MOD", "mod", "Mod"]
            sevTerms = ["SEV", "sev", "Sev"]
            mixTerms = ["MIX", "mix", "Mix", "all", "All", "ALL"]
            hasMild = False
            hasMod = False
            hasSev = False
            hasMix = False
            if any(mild in row['Severity'] for mild in mildTerms):
                hasMild = True
            if any(mod in row['Severity'] for mod in modTerms):  #we sometimes used shortenings for moderate/severe
                hasMod = True
            if any(sev in row['Severity'] for sev in sevTerms):
                hasSev = True
            if any(mix in row['Severity'] for mix in mixTerms):
                hasMix = True
                
            if row['Severity'] == "" and index-1 >= 0:   #fill down
                datalist[i].at[index, 'Severity'] = datalist[i].at[index -1, 'Severity'] #fill down
            elif not row['Severity'] == "" and not any(keyw in row['Severity'] for keyw in mildTerms + modTerms + sevTerms + mixTerms):  #check for errors
                datalist[i].at[index, 'Severity'] = "//ERROR//"
            elif hasMild and hasMod and not hasSev:
                datalist[i].at[index, 'Severity'] = "m/mod"
            elif not hasMild and hasMod and hasSev:
                datalist[i].at[index, 'Severity'] = "mod/sev"
            elif hasMild and not hasMod and not hasSev:
                datalist[i].at[index, 'Severity'] = "mild"
            elif not hasMild and hasMod and not hasSev:
                datalist[i].at[index, 'Severity'] = "moderate"
            elif not hasMild and not hasMod and hasSev:
                datalist[i].at[index, 'Severity'] = "severe"
            elif hasMix:
                datalist[i].at[index, 'Severity'] = "mix severity"
            elif not row['Severity'] == "":
                datalist[i].at[index, 'Severity'] = "mix severity"
            
            
            #clean Age section
            #allows filling down since we wrote this info only once per paper
            adultList = ["ADULT", "adult", 'Adult']
            adolescentList = ["ADOL", "adolescent", "Adolescent"]
            childList = ["CHILD, ""child", "Child"]
            mixList = ["MIX", "mix", "Mix"]
            if any(mix in row['Age'] for mix in mixList) or (any(adult in row['Age'] for adult in adultList) and (any(adolescent in row['Age'] for adolescent in adolescentList) or any(child in row['Age'] for child in childList))) or (any(adolescent in row['Age'] for adolescent in adolescentList) and any(child in row['Age'] for child in childList)):
                datalist[i].at[index, 'Age'] = "mix age"
            elif any(adult in row['Age'] for adult in adultList):
                datalist[i].at[index, 'Age'] = 'adult'
            elif any(adolescent in row['Age'] for adolescent in adolescentList):
                datalist[i].at[index, 'Age'] = "adolescent"
            elif any(child in row['Age'] for child in childList):
                datalist[i].at[index, 'Age'] = "child"
            elif row['Age'] == "" and index-1 >= 0:  #fill down
                datalist[i].at[index, 'Age'] = datalist[i].at[index -1, 'Age'] #fill down
            else:
                datalist[i].at[index, 'Age'] = "//ERROR//"

            #clean Chronicity Section 
            #note: no filling, each finding must have its own chronicity
            acuteList = ["ACUTE", "acute", "Acute"]
            subacuteList = ["SUBACUTE", "subacute", "Subacute"]
            chronicList = ["CHRONIC", "chronic", "Chronic"]
            mixedList = ["MIX", "mix", "Mix"]
            repSubList = ["REP", "SUBCON", "repetitive", "Repetitive", "subconcussive", "Subconcussive"]
            hasAcute = False
            hasSubacute = False
            hasChronic = False
            hasMix = False
            hasRepSub = False
            
            #dealing with the fact that acute exists within subacute
            numAcutes = 0
            for acute in acuteList:
                numAcutes += row['Chronicity'].count(acute) 
            
            #if acute and not subacute we have an actual acute, if acute shows up multiple places we should have acute
            #in the form acute/subacute
            if numAcutes == 1 and not any (sub in row['Chronicity'] for sub in subacuteList) or numAcutes > 1:
                hasAcute = True
                
            if any (sub in row['Chronicity'] for sub in subacuteList):
                hasSubacute = True
            if any (chron in row['Chronicity'] for chron in chronicList):
                hasChronic = True
            if any (mix in row['Chronicity'] for mix in mixedList):
                hasMix = True
            if any (repSub in row['Chronicity'] for repSub in repSubList):
                hasRepSub = True
            if hasRepSub:
                datalist[i].at[index, 'Chronicity'] = "repsub"
            elif hasMix or (hasAcute and hasChronic):
                datalist[i].at[index, 'Chronicity'] = "mix cnicity"
            elif hasAcute and hasSubacute and not hasChronic:
                datalist[i].at[index, 'Chronicity'] = "ac/subac"
            elif not hasAcute and hasSubacute and hasChronic:
                datalist[i].at[index, 'Chronicity'] = "subac/chron"
            elif not hasAcute and not hasSubacute and hasChronic:
                datalist[i].at[index, 'Chronicity'] = "chronic"
            elif hasAcute and not hasSubacute and not hasChronic:
                datalist[i].at[index, 'Chronicity'] = "Acute"
            elif not hasAcute and hasSubacute and not hasChronic:
                datalist[i].at[index, 'Chronicity'] = "subacute"   
            else:
                datalist[i].at[index, 'Chronicity'] = "//ERROR//"
                
            #clean Control Section
            #allows filling down since we wrote this info only once per paper
            healthyList = ["HC", "healthy", "Healthy", "HEALTHY"]
            inSportList = ["ISC", "In-sport", "in-sport", "IN-SPORT", "IN SPORT"]
            nonContactList = ["NCC", "Non-contact", "non-contact", "NON-CONTACT", "non-contact", "Non-Contact", "NON CONTACT"]
            tbiPlusMoodList = ["TBI+", "tbi+", "Tbi+", "TBI plus", "tbi plus", "Tbi plus", "tbi +", "Tbi +", "TBI PLUS"]
            moodList = ["mood", "Mood", "anxiety", "Anxiety", "MOOD", "ANXIETY"]
            otherList =  ["other", "Other", "Orthopedic", "orthopedic", "military", "Military", "OTHER", "ORTHO", "MILIT"]
            if row['Control Type'] == "" and index-1 >= 0:   #fill down
                datalist[i].at[index, 'Control Type'] = datalist[i].at[index -1, 'Control Type']
            elif any(healthy in row['Control Type'] for healthy in healthyList):
                datalist[i].at[index, 'Control Type'] = "HC"
            elif any(inSport in row['Control Type'] for inSport in inSportList):
                datalist[i].at[index, 'Control Type'] = "ISC"
            elif any(nonCon in row['Control Type'] for nonCon in nonContactList):
                datalist[i].at[index, 'Control Type'] = "NCC"    
            elif any(tbiPlus in row['Control Type'] for tbiPlus in tbiPlusMoodList):
                datalist[i].at[index, 'Control Type'] = "TBI+"
            elif any(mood in row['Control Type'] for mood in moodList):
                datalist[i].at[index, 'Control Type'] = "Mood"
            elif any(other in row['Control Type'] for other in otherList):
                datalist[i].at[index, 'Control Type'] = "Other Control"    
            else: 
                datalist[i].at[index, 'Control Type'] = "//ERROR//"
                
    return datalist      
##end cleanData function ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----


#actually clean the data
data = cleanData(data)

#AFTER PROCESSING: 


netLabels = ["DMN", "ECN", "Limbic", "Salience", "DAN", "VAN", "Visual", "Sensorimotor"]
for i in range(len(data)):
    print(netLabels[i] + ":")
    display(data[i])
            
            

DMN:


,Paper,RESULT,TBI Class,Severity,Age,Chronicity,CSX/Long,Control Type,Scanner,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Comp Type,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Mayer (2011) University of New Mexico,dec,civilian,mild,adult,ac/subac,CSX + Long,HC,3 Tesla Siemens Trio scanner,,spatially registered in both two‐ and three‐di...,,funectional connectivity,"seed- based, seeds placed in rACC and PCC for ...",DMN and fronto-parietal task related network,"Decreased BOLD conn within DMN, hyperconn bet...","emotional, cognitive, somatic complaints",,,Seems that the longitudinal portion of the stu...,
1,"Rajesh (2017) Iowa City, Iowa",dec,civilian,mild,adult,chronic,CSX,HC,Siemens Trio 3 Tesla full body magnet,"2 x 2.5 min, eyes closed","""Raw DICOM images were converted to NIfTI form...","FSL 5.0.4, AFNI< MATLAB",FC,seed-based (seeds in rACC and PCC: (12-mm sph...,DMN,"""PCC region in healthy controls showed signifi...",cognitive tasks performed,"""In sum, our results indicate that brain archi...",,,DF
2,,null,civilian,mild,adult,chronic,,HC,,,,,,,,,,,,,
3,Sharp (2011) UK?,inc,civilian,mix severity,adult,chronic,CSX,HC,"MRI data were obtained using a Philips (Best, ...","closed, 10 min",,FMRIB Software Library,connectivity,ICA + Dual Regression,"DMN, frontoparietal, executive, visual, sensor...",TBI Patients demonstrate increased DMN connect...,Greater PCC <-> conn linked to faster reaction...,Increased DMN conn at rest in TBI patients is ...,They mentioned the specific types of injury as...,,
4,"Iraji (2015) Michigan, USA",dec,civilian,mild,adult,Acute,CSX,HC,3-Tesla Siemens Verio,"Eyes closed, 8 mins","First, the first five volumes were discarded t...",FSL,Connectivity,ICA (group and individual) + Seed-based,"ICA: DMN, BGN; SBA: PCC,\nthalamus, amygdala, ...",Both group-based and individual-based independ...,,The study demonstrated multinetwork alteration...,Proposed an atlas-based ICA,,
5,,dec,civilian,mild,adult,Acute,,HC,,,,,,,,,,,,,
6,"van der Horn (2019) Groningen, Netherlands",null,civilian,m/mod,adult,subacute,CSX + Long (93% of patients stayed for long),HC,3 Tesla Philips Intera Achieva MRI‐scanner (P...,closed eyes,"""Volumes were reoriented to the AC, slice time...",SPM12,Static and Dynamic FC,"ROItoROI for static, dynamic: windowed correla...",48 RSNs defined by their ICA. 9 domains: subco...,"""patients with incomplete recovery spent less ...","neural basis of functional outcome, measured b...",,No hyperlink.,,DF
7,,null,civilian,m/mod,adult,subac/chron,,HC,,,,,,,,,,,,,
8,"De Simoni (2016) London, UK",inc,civilian,mod/sev,adult,ac/subac,CSX + Long,HC,"3.0 T GE medical systems scanner, 8-channel he...","7 minutes, eyes open","rigit body realignment, spatial smoothing usin...",FMRIB software library,functional connectivity,"dual regression, seed-based","DMN, MTL",connectivity of PCC to rest of DMN - healthy c...,TBI controls show no difference in fconn btwn ...,,,,
9,,null,civilian,mod/sev,adult,ac/subac,,HC,,,,,,,,,,,,,


ECN:


,Paper,RESULT,TBI Class,Severity,Age,Chronicity,CSX/Long,Control Type,Scanner,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Comp Type,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Mayer (2011) University of New Mexico,inc,civilian,mild,adult,ac/subac,CSX + Long,HC,3 Tesla Siemens Trio scanner,,spatially registered in both two‐ and three‐di...,,funectional connectivity,"seed- based, seeds placed in rACC and PCC for ...",DMN and fronto-parietal task related network,"Decreased BOLD conn within DMN, hyperconn bet...","emotional, cognitive, somatic complaints",,,Seems that the longitudinal portion of the stu...,
1,,inc,civilian,mild,adult,ac/subac,,HC,,,,,,,,,,,,,
2,Sharp (2011) UK?,inc,civilian,mix severity,adult,chronic,CSX,HC,"MRI data were obtained using a Philips (Best, ...","closed, 10 min",,FMRIB Software Library,connectivity,ICA + Dual Regression,"DMN, frontoparietal, executive, visual, sensor...",TBI Patients demonstrate increased DMN connect...,Greater PCC <-> conn linked to faster reaction...,Increased DMN conn at rest in TBI patients is ...,They mentioned the specific types of injury as...,,
3,"van der Horn (2019) Groningen, Netherlands",null,civilian,m/mod,adult,subacute,CSX + Long (93% of patients stayed for long),HC,3 Tesla Philips Intera Achieva MRI‐scanner (P...,closed eyes,,,,,,,,,,,
4,,null,civilian,m/mod,adult,subac/chron,,HC,,,,,,,,,,,,,
5,"van der Horn (2017) #2 Groningen, Netherlands",null,sport,mild,adult,subacute,CSX + Long,HC,,,,,DMN connectivity,,"Default, executive, salience",,Post-traumatic complaints) mTBI group: Increas...,,,,
6,,null,sport,mild,adult,chronic,,HC,,,,,,,,,,,,,
7,"van der Horn (2016) Groningen, Netherlands",null,civilian,mild,adult,subacute,CSX,HC,3.0 T Philips Intera MRI (Phillips Medical Sys...,"closed eyes, ""Three-hundred vol-umes were acqu...","Slice timing, head motion, T1-co registration,...",SPM12,FC (static and dynamic),ROItoROI?,"DMN, Executive (frontoparietal and bilateral f...",No differences in within-network functional co...,"Post-traumatic complaints, anxiety, depression...",,,,DF
8,"Sours (2015) Baltimore, MD",null,civilian,mix severity,adult,Acute,CSX + Long,HC,Siemens Tim-Trio 3T MRI scanner using a 12 cha...,"3m 57s, eyes closed","Motion correction, slice timing correction, ba...",SPM 8,connectivity,ROI to ROI and voxel-based,"DMN, TPN",Iincreased rs- functional connectivity between...,"Test performance, global life satisfaction",,Long findings only significant for CBF (DTI),,
9,,null,civilian,mix severity,adult,subacute,,HC,,,,,,,,,,,,,


Limbic:


...

,Paper,RESULT,TBI Class,Severity,Age,Chronicity,CSX/Long,Control Type,Scanner,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Comp Type,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,"Shumskaya (2012) Nijmegen, Netherlands",null,civilian,mild,adult,ac/subac,CSX,HC,Siemens 3T Trio,6 mins; eyes closed,"head motion correction, extraction of nonbrain...",FMRIB Software Library,connectivity,ICA,"16 networks from ICA incl. DMN, DAN, frontopar...",mTBI Group: Decrease in FC within motor-striat...,Neuropsych performance,whole-brain functional connectivity is altered...,,,
1,"Clough (2018), Victoria, AUS",null,sport,mild,adult,chronic,CSX,NCC,3T Magnetom Prisma,eyes closed; 7 min 35 sec,FSL package and included skull stripping (Brai...,In house pipeline,Connectivity,ROItoROI,40 cortical regions related to seven resting-s...,There were no significant group differences on...,Oculomotor Cognitive Control Abnormalities,,,,HH
2,Roy (2017),null,civilian,mod/sev,adult,chronic,CSX,HC,Philips Achieva 3T or Siemens Magnetom Trio 3T...,unspecified,SPM8 (Also Atlas: Power's 264),ROI to ROI,,,,,,,,,
3,,inc,civilian,mod/sev,adult,chronic,CSX,HC,Philips Achieva 3T or Siemens Magnetom Trio 3T...,unspecified,SPM8 (Also Atlas: Power's 264),ROI to ROI,,,,,,,,,


Visual:


,Paper,RESULT,TBI Class,Severity,Age,Chronicity,CSX/Long,Control Type,Scanner,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Comp Type,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Sharp (2011) UK?,null,civilian,mix severity,adult,chronic,CSX,HC,"MRI data were obtained using a Philips (Best, ...","closed, 10 min",,FMRIB Software Library,connectivity,ICA + Dual Regression,"DMN, frontoparietal, executive, visual, sensor...",TBI Patients demonstrate increased DMN connect...,Greater PCC <-> conn linked to faster reaction...,Increased DMN conn at rest in TBI patients is ...,They mentioned the specific types of injury as...,,
1,"van der Horn (2019) Groningen, Netherlands",null,civilian,m/mod,adult,subacute,CSX + Long (93% of patients stayed for long),HC,3 Tesla Philips Intera Achieva MRI‐scanner (P...,closed eyes,,,,,,,,,,,
2,,null,civilian,m/mod,adult,subac/chron,,HC,,,,,,,,,,,,,
3,"Shumskaya (2012) Nijmegen, Netherlands",null,civilian,mild,adult,ac/subac,CSX,HC,Siemens 3T Trio,6 mins; eyes closed,"head motion correction, extraction of nonbrain...",FMRIB Software Library,connectivity,ICA,"16 networks from ICA incl. DMN, DAN, frontopar...",mTBI Group: Decrease in FC within motor-striat...,Neuropsych performance,whole-brain functional connectivity is altered...,,,
4,"Stevens (2012) Hartford, CT",dec,civilian,mild,adult,subac/chron,CSX,HC,Siemens Allegra 3T,"open, fixed gaze, 5:15 min",Rotational/Translational Correction + head cus...,GIFT v1.3i toolbox implemented in Matlab,connectivity,"Group ICA, novel ""multivariate"" technique","Ventromedial Cortex Network, Visual, Limbic, C...",mTBI patients demonstragte significant differe...,Post-concussive syndrome complaints,,"Pretty extensive findings, seems broad/overzea...",,
5,,inc,civilian,mild,adult,subac/chron,,HC,,,,,,,,,,,,,
6,,inc,civilian,mild,adult,subac/chron,,HC,,,,,,,,,,,,,
7,"Li (2020b) Nanjing, China",dec,civilian,mild,adult,Acute,CSX,HC,"3.0 T MRI scanner (Ingenia, Philips)","8 min and 6 s, eyes closed",slice-timing corrected and calibrated for head...,DPABI_ V4.3_200301,FC,ICA,"Among the 34 components resulting from ICA, se...",WITHIN: Significant differences between the mT...,d,,,,
8,"Vakhtin (2016) Albuquerque, NM",null,military,mild,adult,//ERROR//,CSX,HC,3-Tesla Siemens Trio,"5m 34s, eyes open on cross",same as Allen (2011),same as Allen (2011),FC mostly. Specifically.. 3 fMRI domains: bloo...,ICA (GIFT software),"28 networks across brain (6 sensorimotor, 6 at...","""The mTBI group exhibited hyperactivity in the...","""The following neuropsychological tests were a...","""The results suggest white matter disruption a...",purely blast wave encounters for the injuries,,
9,"Grossner (2019), Pennsylvania, USA",null,civilian,mod/sev,adult,subac/chron,CSX,HC,2: Philips Achieve 3T scanner or a Siemens\nMa...,"10 mins, fixate on a white\ncross in the cente...","""All preprocessing steps used SPM8. Using ArtR...",SPM8,within and between network functional connecti...,Graph,First used a functionally defined atlas (Power...,TBI Group had signifantly higher intranetwork ...,Metacognitive accuracy: Significant interactio...,Enhanced connectivity to both anterior and pos...,,,


Sensorimotor:


,Paper,RESULT,TBI Class,Severity,Age,Chronicity,CSX/Long,Control Type,Scanner,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Comp Type,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Sharp (2011) UK?,inc,civilian,mix severity,adult,chronic,CSX,HC,"MRI data were obtained using a Philips (Best, ...","closed, 10 min",,FMRIB Software Library,connectivity,ICA + Dual Regression,"DMN, frontoparietal, executive, visual, sensor...",TBI Patients demonstrate increased DMN connect...,Greater PCC <-> conn linked to faster reaction...,Increased DMN conn at rest in TBI patients is ...,They mentioned the specific types of injury as...,,
1,"van der Horn (2019) Groningen, Netherlands",null,civilian,m/mod,adult,subacute,CSX + Long (93% of patients stayed for long),HC,3 Tesla Philips Intera Achieva MRI‐scanner (P...,closed eyes,,,,,,,,,,,
2,,null,civilian,m/mod,adult,subac/chron,,HC,,,,,,,,,,,,,
3,"Nathan (2015) Bethesda, MD USA",inc,military,mild,adult,subac/chron,CSX,HC,"3 T GE Healthcare, 32 channel","6 min, eyes closed",3D volume registrations to correct head moveme...,"AFNI, FSL package, MELODIC tool",FC,ICA,DMN,mTBI subjects had decrease in DMN connectivity...,"GOF analysis, ANOVA failed to show a significa...",,,,
4,"Shumskaya (2012) Nijmegen, Netherlands",dec,civilian,mild,adult,ac/subac,CSX,HC,Siemens 3T Trio,6 mins; eyes closed,"head motion correction, extraction of nonbrain...",FMRIB Software Library,connectivity,ICA,"16 networks from ICA incl. DMN, DAN, frontopar...",mTBI Group: Decrease in FC within motor-striat...,Neuropsych performance,whole-brain functional connectivity is altered...,,,
5,,null,civilian,mild,adult,ac/subac,,HC,,,,,,,,,,,,,
6,"Stevens (2012) Hartford, CT",dec,civilian,mild,adult,subac/chron,CSX,HC,Siemens Allegra 3T,"open, fixed gaze, 5:15 min",Rotational/Translational Correction + head cus...,GIFT v1.3i toolbox implemented in Matlab,connectivity,"Group ICA, novel ""multivariate"" technique","Ventromedial Cortex Network, Visual, Limbic, C...",mTBI patients demonstragte significant differe...,Post-concussive syndrome complaints,,"Pretty extensive findings, seems broad/overzea...",,
7,,inc,civilian,mild,adult,subac/chron,,HC,,,,,,,,,,,,,
8,"Rigon (2016) IA, USA",null,civilian,mix severity,adult,chronic,CSX,HC,Siemens Magneto TIM Trio 3T,"6min, eyes closed","motion correction, spatial smoothing",FMRIB Software Library - FSL 5.0.4,connectivity,Seed-based,"Fronto-parietal network (FPN), DMN, Executive ...",(1) Significantly less inter-hemispheric FC in...,Neuropsych performance (ROCFT),,,,
9,"Li (2020b) Nanjing, China",null,civilian,mild,adult,Acute,CSX,HC,"3.0 T MRI scanner (Ingenia, Philips)","8 min and 6 s, eyes closed",slice-timing corrected and calibrated for head...,DPABI_ V4.3_200301,FC,ICA,"Among the 34 components resulting from ICA, se...",WITHIN: Significant differences between the mT...,d,,,,HH


In [36]:
#COUNT EVERYTHING UP AND REPORT IT!

classifications = []   #to hold all possible specific classifications

#made it so that no string was contained within another string to make for simpler searching when coming up with statistics
nets = ["dmn", "ecn", "limb", "sn", "dan", "van", "vis", "smn"]
results = ["inc", "dec", "null"]
severities = ["mild", "m/mod", "moderate", "mod/sev", "severe", "mix severity", "No Severity"]
ages = ["child", "adolescent", "adult", "mix age", "No Age"]
chronicities = ["Acute", "ac/subac", "subacute", "subac/chron", "chronic", "repsub", "mix cnicity", "No Cnicity"]
types = ["sport", "military", "civilian", "mix Type"]
controls = ["HC", "ISC", "NCC", "TBI+", "Mood", "Other Control"]

for net in nets:
    for result in results:
        for severity in severities:
            for age in ages:
                for chronicity in chronicities:
                    for typ in types:
                        for control in controls:
                            classifications.append(net + "_" + result + "_" + severity + "_" + age +
                                                   "_" + chronicity + "_" + typ + "_" + control)

#print(len(classifications)) 

#errors should already be checked! they can be allowed to 
#remain if desired for missing info on severity, age, chronicity. Other rows must be figured out. Data is required.
def runStats(datalist, stats):
    for i in range(len(datalist)):
        for index, row in datalist[i].iterrows():
            classif = ""
            classif += dataOrder[i]
            classif += "_" + row['RESULT']
            
            
            if row['Severity'] == "//ERROR//":
                classif += "_" + "No Severity"
            else:
                classif += "_" + row['Severity']
                
            if row['Age'] == "//ERROR//":
                classif += "_" + "No Age"
            else:
                classif += "_" + row['Age']
                
            if row['Chronicity'] == "//ERROR//":
                classif += "_" + "No Cnicity"
            else:
                classif += "_" + row['Chronicity']
                
            classif += "_" + row['TBI Class']
            classif += "_" + row['Control Type']
            
            for stat in stats:
                if stat[0] == classif:
                    stat[1] += 1
                    
stats = [[classi, 0] for classi in classifications]            
runStats(data, stats)

#stats

#input up to 6 queries (ie: queryStats("dmn", "mild") and get a count back of the number of inc, dec, null results)
def queryStats(stats, query1="", query2="", query3="", query4="", query5="", query6=""):
    count_inc = 0
    count_dec = 0
    count_null = 0
    for stat in stats:
        if "inc" in stat[0] and query1 in stat[0] and query2 in stat[0] and query3 in stat[0] \
            and query4 in stat[0] and query5 in stat[0] and query6 in stat[0]:
                count_inc += stat[1]
        if "dec" in stat[0] and query1 in stat[0] and query2 in stat[0] and query3 in stat[0] \
            and query4 in stat[0] and query5 in stat[0] and query6 in stat[0]:
                count_dec += stat[1]
        if "null" in stat[0] and query1 in stat[0] and query2 in stat[0] and query3 in stat[0] \
            and query4 in stat[0] and query5 in stat[0] and query6 in stat[0]:
                count_null += stat[1]
    return [count_inc, count_dec, count_null]

def printStats(stats):
    print('Starting Counts...')
    print('\n')
    
    #network totals:
    
    #dmn
    dmn = queryStats(stats, "dmn")

    #ecn        
    ecn = queryStats(stats, "ecn")
    
    #Limbic
    limb = queryStats(stats, "limb")
    
    #Salience/Ventral
    sn = queryStats(stats, "sn")
    
    van = queryStats(stats, "van")
            
    snvan = [sn[0] + van[0], sn[1] + van[1], sn[2] + van[2]]
    
    
    #DAN
    dan = queryStats(stats, "dan")
    
    #Visual
    vis = queryStats(stats, "vis")
    
    #Sensorimotor
    smn = queryStats(stats, "smn")
    
    total_inc = dmn[0] + ecn[0] + limb[0] + snvan[0] + dan[0] + vis[0] + smn[0]
    total_dec = dmn[1] + ecn[1] + limb[1] + snvan[1] + dan[1] + vis[1] + smn[1]
    total_null = dmn[2] + ecn[2] + limb[2] + snvan[2] + dan[2] + vis[2] + smn[2]
    
    data_totals = {'Increase':[dmn[0], ecn[0], limb[0], snvan[0], dan[0], vis[0], smn[0], total_inc], 
                'Decrease':[dmn[1], ecn[1], limb[1], snvan[1], dan[1], vis[1], smn[1], total_dec], 
                'Null':[dmn[2], ecn[2], limb[2], snvan[2], dan[2], vis[2], smn[2], total_null]}        
    df_totals = pd.DataFrame(data_totals, index = ['DMN', 'ECN', 'Limbic', 'Salience/VAN', 'DAN', 'Visual', 'Sensorimotor', 'Total'])
    
    print("Network Totals: ")
    display(df_totals)
    
    #now in detail..
    detailed_index = ['All', 'Mild', 'Mild/Moderate', 'Moderate', 'Moderate/Severe', 
             'Severe', 'Mixed Severity', 'No Severity Stated', 'Child', 'Adolescent', 'Adult', 'Mixed Age', 'No Age Stated', 
             'Acute', 'Acute/Subacute', 'Subacute', 'Subacute/Chronic', 'Chronic', 'Repetitive Subconcussive',
             'Mixed Chronicity', 'No Chronicity Stated', 'Sport', 'Military', 'Civilian', 'Mixed Type', 
             'Healthy Control', 'No Contact Control', 'In Sport Control', 'Other Control']
    
    
    #All networks in Detail-------------------:
    #nets = ["dmn", "ecn", "limb", "sn", "dan", "van", "vis", "smn"]
#results = ["inc", "dec", "null"]
#severities = ["mild", "m/mod", "moderate", "mod/sev", "severe", "mix severity", "No Severity"]
#ages = ["child", "adolescent", "adult", "mix age", "No Age"]
#chronicities = ["Acute", "ac/subac", "subacute", "subac/chron", "chronic", "repsub", "mix cnicity", "No Cnicity"]
#types = ["sport", "military", "civilian", "mix Type"]
#controls = ["HC", "ISC", "NCC", "TBI+", "Mood", "Other Control"]
    print("All Networks in Detail:")

    all = [total_inc, total_dec, total_null]
    all_mild = queryStats(stats, "mild")
    all_mildMod = queryStats(stats, "m/mod")
    all_mod = queryStats(stats, "moderate")
    all_modSev = queryStats(stats, "mod/sev")
    all_sev = queryStats(stats, "severe")
    all_mixSev = queryStats(stats, "mix severity")
    all_noSev = queryStats(stats, "No Severity")
    all_child = queryStats(stats, "child")
    all_adolescent = queryStats(stats, "adolescent")
    all_adult = queryStats(stats, "adult")
    all_mixAge = queryStats(stats, "mix age")
    all_noAge = queryStats(stats, "No Age")
    all_acute = queryStats(stats, "Acute")
    all_acuteSub = queryStats(stats, "ac/subac")
    all_subacute = queryStats(stats, "subacute")
    all_subChron = queryStats(stats, "subac/chron")
    all_chronic = queryStats(stats, "chronic")
    all_repSub = queryStats(stats, "repsub")
    all_mixChron = queryStats(stats, "mix cnicity")
    all_noChron = queryStats(stats, "No Cnicity")
    all_sport = queryStats(stats, "sport")
    all_military = queryStats(stats, "military")
    all_civilian = queryStats(stats, "civilian")
    all_mixType = queryStats(stats, "mix Type")
    all_healthyControl = queryStats(stats, "HC")
    all_noContactControl = queryStats(stats, "NCC")
    all_inSportControl = queryStats(stats, "ISC")
    all_otherControl = queryStats(stats, "Other Control")
    
    all_categories = [all, all_mild, all_mildMod, all_mod, all_modSev, all_sev, 
                  all_mixSev, all_noSev, all_child, all_adolescent, all_adult,
                 all_mixAge, all_noAge, all_acute, all_acuteSub, all_subacute, all_subChron, all_chronic, all_repSub, 
                 all_mixChron, all_noChron, all_sport, all_military, all_civilian, all_mixType, 
                 all_healthyControl, all_noContactControl, all_inSportControl, all_otherControl]
    
    all_incList = []
    all_decList = []
    all_nullList = []
    for cat in all_categories:
        all_incList.append(cat[0])
        all_decList.append(cat[1])
        all_nullList.append(cat[2])
    
    data_all = {'Increase': all_incList, 
                'Decrease': all_decList, 
                'Null': all_nullList}        
    df_all = pd.DataFrame(data_all, index = detailed_index)
    display(df_all)
    
    #BY SEVERITY AND CHRONICITY (ALL)
    print("Results by Severity & Chronicity")
    mildAcute = queryStats(stats, "mild", "Acute")
    mildAcuteSubacute = queryStats(stats, "mild", "ac/subac")
    mildSubacute = queryStats(stats, "mild", "subacute")
    mildSubacuteChronic = queryStats(stats, "mild", "subac/chron")
    mildChronic = queryStats(stats, "mild", "chronic")
    mildMixChron = queryStats(stats, "mild", "mix cnicity")
    mildNoChron = queryStats(stats, "mild", "No Cnicity")
    
    mildModAcute = queryStats(stats, "m/mod", "Acute") 
    mildModAcuteSubacute = queryStats(stats, "m/mod", "ac/subac")
    mildModSubacute = queryStats(stats, "m/mod", "subacute")
    mildModSubacuteChronic = queryStats(stats, "m/mod", "subac/chron")
    mildModChronic = queryStats(stats, "m/mod", "chronic")
    mildModMixChron = queryStats(stats, "m/mod", "mix cnicity")
    mildModNoChron = queryStats(stats, "m/mod", "No Cnicity")
    
    modAcute = queryStats(stats, "moderate", "Acute") 
    modAcuteSubacute = queryStats(stats, "moderate", "ac/subac")
    modSubacute = queryStats(stats, "moderate", "subacute")
    modSubacuteChronic = queryStats(stats, "moderate", "subac/chron")
    modChronic = queryStats(stats, "moderate", "chronic")
    modMixChron = queryStats(stats, "moderate", "mix cnicity")
    modNoChron = queryStats(stats, "moderate", "No Cnicity")
    
    modSevereAcute = queryStats(stats, "mod/sev", "Acute") 
    modSevereAcuteSubacute = queryStats(stats, "mod/sev", "ac/subac")
    modSevereSubacute = queryStats(stats, "mod/sev", "subacute")
    modSevereSubacuteChronic = queryStats(stats, "mod/sev", "subac/chron")
    modSevereChronic = queryStats(stats, "mod/sev", "chronic")
    modSevereMixChron = queryStats(stats, "mod/sev", "mix cnicity")
    modSevereNoChron = queryStats(stats, "mod/sev", "No Cnicity")
    
    severeAcute = queryStats(stats, "severe", "Acute") 
    severeAcuteSubacute = queryStats(stats, "severe", "ac/subac")
    severeSubacute = queryStats(stats, "severe", "subacute")
    severeSubacuteChronic = queryStats(stats, "severe", "subac/chron")
    severeChronic = queryStats(stats, "severe", "chronic")
    severeMixChron = queryStats(stats, "severe", "mix cnicity")
    severeNoChron = queryStats(stats, "severe", "No Cnicity")
    
    mixedSevAcute = queryStats(stats, "mix severity", "Acute") 
    mixedSevAcuteSubacute = queryStats(stats, "mix severity", "ac/subac")
    mixedSevSubacute = queryStats(stats, "mix severity", "subacute")
    mixedSevSubacuteChronic = queryStats(stats, "mix severity", "subac/chron")
    mixedSevChronic = queryStats(stats, "mix severity", "chronic")
    mixedSevMixChron = queryStats(stats, "mix severity", "mix cnicity")
    mixedSevNoChron = queryStats(stats, "mix severity", "No Cnicity")
    
    noSevAcute = queryStats(stats, "No Severity", "Acute") 
    noSevAcuteSubacute = queryStats(stats, "No Severity", "ac/subac")
    noSevSubacute = queryStats(stats, "No Severity", "subacute")
    noSevSubacuteChronic = queryStats(stats, "No Severity", "subac/chron")
    noSevChronic = queryStats(stats, "No Severity", "chronic")
    noSevMixChron = queryStats(stats, "No Severity", "mix cnicity")
    noSevNoChron = queryStats(stats, "No Severity", "No Cnicity")
    
    sevChron_list = [mildAcute, mildAcuteSubacute, mildSubacute, mildSubacuteChronic, mildChronic, mildMixChron,
                    mildNoChron, mildModAcute, mildModAcuteSubacute, mildModSubacute, mildModSubacuteChronic, 
                    mildModChronic, mildModMixChron, mildModNoChron, modAcute, modAcuteSubacute, modSubacute,
                    modSubacuteChronic, modChronic, modMixChron, modNoChron, modSevereAcute, modSevereAcuteSubacute,
                    modSevereSubacute, modSevereSubacuteChronic, modSevereChronic, modSevereMixChron, modSevereNoChron,
                    severeAcute, severeAcuteSubacute, severeSubacute, severeSubacuteChronic, severeChronic, severeMixChron,
                    severeNoChron, mixedSevAcute, mixedSevAcuteSubacute, mixedSevSubacute, mixedSevSubacuteChronic,
                    mixedSevChronic, mixedSevMixChron, mixedSevNoChron, noSevAcute, noSevAcuteSubacute, noSevSubacute, noSevSubacuteChronic,
                    noSevChronic, noSevMixChron, noSevNoChron]
    
    sevChron_incList = []
    sevChron_decList = []
    sevChron_nullList = []
    
    for sevChron in sevChron_list:
        sevChron_incList.append(sevChron[0])
        sevChron_decList.append(sevChron[1])
        sevChron_nullList.append(sevChron[2])
    
    sevChron_index = ["Mild Acute", "Mild Acute/Subacute", "Mild Subacute", "Mild Subacute/Chronic", "Mild Chronic", "Mild Mixed Chronicity", "Mild + No Chronicity Stated", 
                      "Mild/Mod Acute", "Mild/Mod Acute/Subacute", "Mild/Mod Subacute", "Mild/Mod Subacute/Chronic", "Mild/Mod Chronic", "Mild/Mod Mixed Chronicity", "Mild/Mod + No Chronicity Stated", 
                      "Moderate Acute", "Moderate Acute/Subacute", "Moderate Subacute", "Moderate Subacute/Chronic", "Moderate Chronic", "Moderate Mixed Chronicity", "Moderate + No Chronicity Stated",
                      "Mod/Sev Acute", "Mod/Sev Acute/Subacute", "Mod/Sev Subacute", "Mod/Sev Subacute/Chronic", "Mod/Sev Chronic", "Mod/Sev Mixed Chronicity", "Mod/Sev + No Chronicity Stated", 
                      "Severe Acute", "Severe Acute/Subacute", "Severe Subacute", "Severe Subacute/Chronic", "Severe Chronic", "Severe Mixed Chronicity", "Severe + No Chronicity Stated",
                      "Mixed Sev. Acute", "Mixed Sev. Acute/Subacute", "Mixed Sev. Subacute", "Mixed Sev. Subacute/Chronic", "Mixed Sev. Chronic", "Mixed Sev. Mixed Chronicity", "Mixed Sev. + No Chronicity Stated", 
                      "No Sev. Stated Acute", "No Sev. Stated Acute/Subacute", "No Sev. Stated Subacute", "No Sev. Stated Subacute/Chronic", "No Sev. Stated Chronic", "No Sev. Stated Mixed Chronicity", "No Sev. Stated + No Chronicity Stated"] 
    
    data_sevChron = {'Increase': sevChron_incList, 
                'Decrease': sevChron_decList, 
                'Null': sevChron_nullList}        
    
    df_sevChron = pd.DataFrame(data_sevChron, sevChron_index)
    display(df_sevChron)
    
    #DMN------------------------------:
    print("DMN in Detail:")
    
    dmn_mild = queryStats(stats, "dmn", "mild")
    dmn_mildMod = queryStats(stats, "dmn", "m/mod")
    dmn_mod = queryStats(stats, "dmn", "moderate")
    dmn_modSev = queryStats(stats, "dmn", "mod/sev")
    dmn_sev = queryStats(stats, "dmn", "severe")
    dmn_mixSev = queryStats(stats, "dmn", "mix severity")
    dmn_noSev = queryStats(stats, "dmn", "No Severity")
    dmn_child = queryStats(stats, "dmn", "child")
    dmn_adolescent = queryStats(stats, "dmn", "adolescent")
    dmn_adult = queryStats(stats, "dmn", "adult")
    dmn_mixAge = queryStats(stats, "dmn", "mix age")
    dmn_noAge = queryStats(stats, "dmn", "No Age")
    dmn_acute = queryStats(stats, "dmn", "Acute")
    dmn_acuteSub = queryStats(stats, "dmn", "ac/subac")
    dmn_subacute = queryStats(stats, "dmn", "subacute")
    dmn_subChron = queryStats(stats, "dmn", "subac/chron")
    dmn_chronic = queryStats(stats, "dmn", "chronic")
    dmn_repSub = queryStats(stats, "dmn", "repsub")
    dmn_mixChron = queryStats(stats, "dmn", "mix cnicity")
    dmn_noChron = queryStats(stats, "dmn", "No Cnicity")
    dmn_sport = queryStats(stats, "dmn", "sport")
    dmn_military = queryStats(stats, "dmn", "military")
    dmn_civilian = queryStats(stats, "dmn", "civilian")
    dmn_mixType = queryStats(stats, "dmn", "mix Type")
    dmn_healthyControl = queryStats(stats, "dmn", "HC")
    dmn_noContactControl = queryStats(stats, "dmn", "NCC")
    dmn_inSportControl = queryStats(stats, "dmn", "ISC")
    dmn_otherControl = queryStats(stats, "dmn", "Other Control")
    
    dmn_categories = [dmn, dmn_mild, dmn_mildMod, dmn_mod, dmn_modSev, dmn_sev, 
                  dmn_mixSev, dmn_noSev, dmn_child, dmn_adolescent, dmn_adult,
                 dmn_mixAge, dmn_noAge, dmn_acute, dmn_acuteSub, dmn_subacute, dmn_subChron, dmn_chronic, dmn_repSub, 
                 dmn_mixChron, dmn_noChron, dmn_sport, dmn_military, dmn_civilian, dmn_mixType, 
                 dmn_healthyControl, dmn_noContactControl, dmn_inSportControl, dmn_otherControl]
    
    dmn_incList = []
    for cat in dmn_categories:
        dmn_incList.append(cat[0])
    dmn_decList = []
    for cat in dmn_categories:
        dmn_decList.append(cat[1])
    dmn_nullList = []
    for cat in dmn_categories:
        dmn_nullList.append(cat[2])
    
    data_dmn = {'Increase': dmn_incList, 
                'Decrease': dmn_decList, 
                'Null': dmn_nullList}        
    df_dmn = pd.DataFrame(data_dmn, index = detailed_index)
    display(df_dmn)
    
    #ECN------------------------------:
    print("ECN in Detail:")
    
    ecn_mild = queryStats(stats, "ecn", "mild")
    ecn_mildMod = queryStats(stats, "ecn", "m/mod")
    ecn_mod = queryStats(stats, "ecn", "moderate")
    ecn_modSev = queryStats(stats, "ecn", "mod/sev")
    ecn_sev = queryStats(stats, "ecn", "severe")
    ecn_mixSev = queryStats(stats, "ecn", "mix severity")
    ecn_noSev = queryStats(stats, "ecn", "No Severity")
    ecn_child = queryStats(stats, "ecn", "child")
    ecn_adolescent = queryStats(stats, "ecn", "adolescent")
    ecn_adult = queryStats(stats, "ecn", "adult")
    ecn_mixAge = queryStats(stats, "ecn", "mix age")
    ecn_noAge = queryStats(stats, "ecn", "No Age")
    ecn_acute = queryStats(stats, "ecn", "Acute")
    ecn_acuteSub = queryStats(stats, "ecn", "ac/subac")
    ecn_subacute = queryStats(stats, "ecn", "subacute")
    ecn_subChron = queryStats(stats, "ecn", "subac/chron")
    ecn_chronic = queryStats(stats, "ecn", "chronic")
    ecn_repSub = queryStats(stats, "ecn", "repsub")
    ecn_mixChron = queryStats(stats, "ecn", "mix cnicity")
    ecn_noChron = queryStats(stats, "ecn", "No Cnicity")
    ecn_sport = queryStats(stats, "ecn", "sport")
    ecn_military = queryStats(stats, "ecn", "military")
    ecn_civilian = queryStats(stats, "ecn", "civilian")
    ecn_mixType = queryStats(stats, "ecn", "mix Type")
    ecn_healthyControl = queryStats(stats, "ecn", "HC")
    ecn_noContactControl = queryStats(stats, "ecn", "NCC")
    ecn_inSportControl = queryStats(stats, "ecn", "ISC")
    ecn_otherControl = queryStats(stats, "ecn", "Other Control")
    
    ecn_categories = [ecn, ecn_mild, ecn_mildMod, ecn_mod, ecn_modSev, ecn_sev, 
                  ecn_mixSev, ecn_noSev, ecn_child, ecn_adolescent, ecn_adult,
                 ecn_mixAge, ecn_noAge, ecn_acute, ecn_acuteSub, ecn_subacute, ecn_subChron, ecn_chronic, ecn_repSub, 
                 ecn_mixChron, ecn_noChron, ecn_sport, ecn_military, ecn_civilian, ecn_mixType, 
                 ecn_healthyControl, ecn_noContactControl, ecn_inSportControl, ecn_otherControl]
    
    ecn_incList = []
    for cat in ecn_categories:
        ecn_incList.append(cat[0])
    ecn_decList = []
    for cat in ecn_categories:
        ecn_decList.append(cat[1])
    ecn_nullList = []
    for cat in ecn_categories:
        ecn_nullList.append(cat[2])
    
    data_ecn = {'Increase': ecn_incList, 
                'Decrease': ecn_decList, 
                'Null': ecn_nullList}        
    df_ecn = pd.DataFrame(data_ecn, index = detailed_index)
    display(df_ecn)
    
    #Limbic------------------------------:
    print("Limbic in Detail:")
    
    limb_mild = queryStats(stats, "limb", "mild")
    limb_mildMod = queryStats(stats, "limb", "m/mod")
    limb_mod = queryStats(stats, "limb", "moderate")
    limb_modSev = queryStats(stats, "limb", "mod/sev")
    limb_sev = queryStats(stats, "limb", "severe")
    limb_mixSev = queryStats(stats, "limb", "mix severity")
    limb_noSev = queryStats(stats, "limb", "No Severity")
    limb_child = queryStats(stats, "limb", "child")
    limb_adolescent = queryStats(stats, "limb", "adolescent")
    limb_adult = queryStats(stats, "limb", "adult")
    limb_mixAge = queryStats(stats, "limb", "mix age")
    limb_noAge = queryStats(stats, "limb", "No Age")
    limb_acute = queryStats(stats, "limb", "Acute")
    limb_acuteSub = queryStats(stats, "limb", "ac/subac")
    limb_subacute = queryStats(stats, "limb", "subacute")
    limb_subChron = queryStats(stats, "limb", "subac/chron")
    limb_chronic = queryStats(stats, "limb", "chronic")
    limb_repSub = queryStats(stats, "limb", "repsub")
    limb_mixChron = queryStats(stats, "limb", "mix cnicity")
    limb_noChron = queryStats(stats, "limb", "No Cnicity")
    limb_sport = queryStats(stats, "limb", "sport")
    limb_military = queryStats(stats, "limb", "military")
    limb_civilian = queryStats(stats, "limb", "civilian")
    limb_mixType = queryStats(stats, "limb", "mix Type")
    limb_healthyControl = queryStats(stats, "limb", "HC")
    limb_noContactControl = queryStats(stats, "limb", "NCC")
    limb_inSportControl = queryStats(stats, "limb", "ISC")
    limb_otherControl = queryStats(stats, "limb", "Other Control")
    
    limb_categories = [limb, limb_mild, limb_mildMod, limb_mod, limb_modSev, limb_sev, 
                  limb_mixSev, limb_noSev, limb_child, limb_adolescent, limb_adult,
                 limb_mixAge, limb_noAge, limb_acute, limb_acuteSub, limb_subacute, limb_subChron, limb_chronic, limb_repSub, 
                 limb_mixChron, limb_noChron, limb_sport, limb_military, limb_civilian, limb_mixType, 
                 limb_healthyControl, limb_noContactControl, limb_inSportControl, limb_otherControl]
    
    limb_incList = []
    for cat in limb_categories:
        limb_incList.append(cat[0])
    limb_decList = []
    for cat in limb_categories:
        limb_decList.append(cat[1])
    limb_nullList = []
    for cat in limb_categories:
        limb_nullList.append(cat[2])
    
    data_limb = {'Increase': limb_incList, 
                'Decrease': limb_decList, 
                'Null': limb_nullList}        
    df_limb = pd.DataFrame(data_limb, index = detailed_index)
    display(df_limb)
    
    #Salience/VAN------------------------:
    print("Salience/VAN in Detail:")
    
    #Salience
    sn_mild = queryStats(stats, "sn", "mild")
    sn_mildMod = queryStats(stats, "sn", "m/mod")
    sn_mod = queryStats(stats, "sn", "moderate")
    sn_modSev = queryStats(stats, "sn", "mod/sev")
    sn_sev = queryStats(stats, "sn", "severe")
    sn_mixSev = queryStats(stats, "sn", "mix severity")
    sn_noSev = queryStats(stats, "sn", "No Severity")
    sn_child = queryStats(stats, "sn", "child")
    sn_adolescent = queryStats(stats, "sn", "adolescent")
    sn_adult = queryStats(stats, "sn", "adult")
    sn_mixAge = queryStats(stats, "sn", "mix age")
    sn_noAge = queryStats(stats, "sn", "No Age")
    sn_acute = queryStats(stats, "sn", "Acute")
    sn_acuteSub = queryStats(stats, "sn", "ac/subac")
    sn_subacute = queryStats(stats, "sn", "subacute")
    sn_subChron = queryStats(stats, "sn", "subac/chron")
    sn_chronic = queryStats(stats, "sn", "chronic")
    sn_repSub = queryStats(stats, "sn", "repsub")
    sn_mixChron = queryStats(stats, "sn", "mix cnicity")
    sn_noChron = queryStats(stats, "sn", "No Cnicity")
    sn_sport = queryStats(stats, "sn", "sport")
    sn_military = queryStats(stats, "sn", "military")
    sn_civilian = queryStats(stats, "sn", "civilian")
    sn_mixType = queryStats(stats, "sn", "mix Type")
    sn_healthyControl = queryStats(stats, "sn", "HC")
    sn_noContactControl = queryStats(stats, "sn", "NCC")
    sn_inSportControl = queryStats(stats, "sn", "ISC")
    sn_otherControl = queryStats(stats, "sn", "Other Control")
    
    sn_categories = [sn, sn_mild, sn_mildMod, sn_mod, sn_modSev, sn_sev, 
                  sn_mixSev, sn_noSev, sn_child, sn_adolescent, sn_adult,
                 sn_mixAge, sn_noAge, sn_acute, sn_acuteSub, sn_subacute, sn_subChron, sn_chronic, sn_repSub, 
                 sn_mixChron, sn_noChron, sn_sport, sn_military, sn_civilian, sn_mixType, 
                 sn_healthyControl, sn_noContactControl, sn_inSportControl, sn_otherControl]
    
    sn_incList = []
    for cat in sn_categories:
        sn_incList.append(cat[0])
    sn_decList = []
    for cat in sn_categories:
        sn_decList.append(cat[1])
    sn_nullList = []
    for cat in sn_categories:
        sn_nullList.append(cat[2])
    
    #VAN
    van_mild = queryStats(stats, "van", "mild")
    van_mildMod = queryStats(stats, "van", "m/mod")
    van_mod = queryStats(stats, "van", "moderate")
    van_modSev = queryStats(stats, "van", "mod/sev")
    van_sev = queryStats(stats, "van", "severe")
    van_mixSev = queryStats(stats, "van", "mix severity")
    van_noSev = queryStats(stats, "van", "No Severity")
    van_child = queryStats(stats, "van", "child")
    van_adolescent = queryStats(stats, "van", "adolescent")
    van_adult = queryStats(stats, "van", "adult")
    van_mixAge = queryStats(stats, "van", "mix age")
    van_noAge = queryStats(stats, "van", "No Age")
    van_acute = queryStats(stats, "van", "Acute")
    van_acuteSub = queryStats(stats, "van", "ac/subac")
    van_subacute = queryStats(stats, "van", "subacute")
    van_subChron = queryStats(stats, "van", "subac/chron")
    van_chronic = queryStats(stats, "van", "chronic")
    van_repSub = queryStats(stats, "van", "repsub")
    van_mixChron = queryStats(stats, "van", "mix cnicity")
    van_noChron = queryStats(stats, "van", "No Cnicity")
    van_sport = queryStats(stats, "van", "sport")
    van_military = queryStats(stats, "van", "military")
    van_civilian = queryStats(stats, "van", "civilian")
    van_mixType = queryStats(stats, "van", "mix Type")
    van_healthyControl = queryStats(stats, "van", "HC")
    van_noContactControl = queryStats(stats, "van", "NCC")
    van_inSportControl = queryStats(stats, "van", "ISC")
    van_otherControl = queryStats(stats, "van", "Other Control")
    
    van_categories = [van, van_mild, van_mildMod, van_mod, van_modSev, van_sev, 
                  van_mixSev, van_noSev, van_child, van_adolescent, van_adult,
                 van_mixAge, van_noAge, van_acute, van_acuteSub, van_subacute, van_subChron, van_chronic, van_repSub, 
                 van_mixChron, van_noChron, van_sport, van_military, van_civilian, van_mixType, 
                 van_healthyControl, van_noContactControl, van_inSportControl, van_otherControl]
    
    van_incList = []
    for cat in van_categories:
        van_incList.append(cat[0])
    van_decList = []
    for cat in van_categories:
        van_decList.append(cat[1])
    van_nullList = []
    for cat in van_categories:
        van_nullList.append(cat[2])
        
    #SN/VAN
    snvan_incList = [a + b for a, b in zip(sn_incList, van_incList)]
    snvan_decList = [a + b for a, b in zip(sn_decList, van_decList)]
    snvan_nullList = [a + b for a, b in zip(sn_nullList, van_nullList)]
    
    data_snvan = {'Increase': snvan_incList, 
                'Decrease': snvan_decList, 
                'Null': snvan_nullList}        
    df_snvan = pd.DataFrame(data_snvan, index = detailed_index)
    display(df_snvan)
    
    #DAN------------------------------:
    print("DAN in Detail:")
    
    dan_mild = queryStats(stats, "dan", "mild")
    dan_mildMod = queryStats(stats, "dan", "m/mod")
    dan_mod = queryStats(stats, "dan", "moderate")
    dan_modSev = queryStats(stats, "dan", "mod/sev")
    dan_sev = queryStats(stats, "dan", "severe")
    dan_mixSev = queryStats(stats, "dan", "mix severity")
    dan_noSev = queryStats(stats, "dan", "No Severity")
    dan_child = queryStats(stats, "dan", "child")
    dan_adolescent = queryStats(stats, "dan", "adolescent")
    dan_adult = queryStats(stats, "dan", "adult")
    dan_mixAge = queryStats(stats, "dan", "mix age")
    dan_noAge = queryStats(stats, "dan", "No Age")
    dan_acute = queryStats(stats, "dan", "Acute")
    dan_acuteSub = queryStats(stats, "dan", "ac/subac")
    dan_subacute = queryStats(stats, "dan", "subacute")
    dan_subChron = queryStats(stats, "dan", "subac/chron")
    dan_chronic = queryStats(stats, "dan", "chronic")
    dan_repSub = queryStats(stats, "dan", "repsub")
    dan_mixChron = queryStats(stats, "dan", "mix cnicity")
    dan_noChron = queryStats(stats, "dan", "No Cnicity")
    dan_sport = queryStats(stats, "dan", "sport")
    dan_military = queryStats(stats, "dan", "military")
    dan_civilian = queryStats(stats, "dan", "civilian")
    dan_mixType = queryStats(stats, "dan", "mix Type")
    dan_healthyControl = queryStats(stats, "dan", "HC")
    dan_noContactControl = queryStats(stats, "dan", "NCC")
    dan_inSportControl = queryStats(stats, "dan", "ISC")
    dan_otherControl = queryStats(stats, "dan", "Other Control")
    
    dan_categories = [dan, dan_mild, dan_mildMod, dan_mod, dan_modSev, dan_sev, 
                  dan_mixSev, dan_noSev, dan_child, dan_adolescent, dan_adult,
                 dan_mixAge, dan_noAge, dan_acute, dan_acuteSub, dan_subacute, dan_subChron, dan_chronic, dan_repSub, 
                 dan_mixChron, dan_noChron, dan_sport, dan_military, dan_civilian, dan_mixType, 
                 dan_healthyControl, dan_noContactControl, dan_inSportControl, dan_otherControl]
    
    dan_incList = []
    for cat in dan_categories:
        dan_incList.append(cat[0])
    dan_decList = []
    for cat in dan_categories:
        dan_decList.append(cat[1])
    dan_nullList = []
    for cat in dan_categories:
        dan_nullList.append(cat[2])
    
    data_dan = {'Increase': dan_incList, 
                'Decrease': dan_decList, 
                'Null': dan_nullList}        
    df_dan = pd.DataFrame(data_dan, index = detailed_index)
    display(df_dan)
    
    #Visual------------------------------:
    print("Visual in Detail:")
    
    vis_mild = queryStats(stats, "vis", "mild")
    vis_mildMod = queryStats(stats, "vis", "m/mod")
    vis_mod = queryStats(stats, "vis", "moderate")
    vis_modSev = queryStats(stats, "vis", "mod/sev")
    vis_sev = queryStats(stats, "vis", "severe")
    vis_mixSev = queryStats(stats, "vis", "mix severity")
    vis_noSev = queryStats(stats, "vis", "No Severity")
    vis_child = queryStats(stats, "vis", "child")
    vis_adolescent = queryStats(stats, "vis", "adolescent")
    vis_adult = queryStats(stats, "vis", "adult")
    vis_mixAge = queryStats(stats, "vis", "mix age")
    vis_noAge = queryStats(stats, "vis", "No Age")
    vis_acute = queryStats(stats, "vis", "Acute")
    vis_acuteSub = queryStats(stats, "vis", "ac/subac")
    vis_subacute = queryStats(stats, "vis", "subacute")
    vis_subChron = queryStats(stats, "vis", "subac/chron")
    vis_chronic = queryStats(stats, "vis", "chronic")
    vis_repSub = queryStats(stats, "vis", "repsub")
    vis_mixChron = queryStats(stats, "vis", "mix cnicity")
    vis_noChron = queryStats(stats, "vis", "No Cnicity")
    vis_sport = queryStats(stats, "vis", "sport")
    vis_military = queryStats(stats, "vis", "military")
    vis_civilian = queryStats(stats, "vis", "civilian")
    vis_mixType = queryStats(stats, "vis", "mix Type")
    vis_healthyControl = queryStats(stats, "vis", "HC")
    vis_noContactControl = queryStats(stats, "vis", "NCC")
    vis_inSportControl = queryStats(stats, "vis", "ISC")
    vis_otherControl = queryStats(stats, "vis", "Other Control")
    
    vis_categories = [vis, vis_mild, vis_mildMod, vis_mod, vis_modSev, vis_sev, 
                  vis_mixSev, vis_noSev, vis_child, vis_adolescent, vis_adult,
                 vis_mixAge, vis_noAge, vis_acute, vis_acuteSub, vis_subacute, vis_subChron, vis_chronic, vis_repSub, 
                 vis_mixChron, vis_noChron, vis_sport, vis_military, vis_civilian, vis_mixType, 
                 vis_healthyControl, vis_noContactControl, vis_inSportControl, vis_otherControl]
    
    vis_incList = []
    for cat in vis_categories:
        vis_incList.append(cat[0])
    vis_decList = []
    for cat in vis_categories:
        vis_decList.append(cat[1])
    vis_nullList = []
    for cat in vis_categories:
        vis_nullList.append(cat[2])
    
    data_vis = {'Increase': vis_incList, 
                'Decrease': vis_decList, 
                'Null': vis_nullList}        
    df_vis = pd.DataFrame(data_vis, index = detailed_index)
    display(df_vis)
    
    #SMN------------------------------:
    print("SMN in Detail:")
    
    smn_mild = queryStats(stats, "smn", "mild")
    smn_mildMod = queryStats(stats, "smn", "m/mod")
    smn_mod = queryStats(stats, "smn", "moderate")
    smn_modSev = queryStats(stats, "smn", "mod/sev")
    smn_sev = queryStats(stats, "smn", "severe")
    smn_mixSev = queryStats(stats, "smn", "mix severity")
    smn_noSev = queryStats(stats, "smn", "No Severity")
    smn_child = queryStats(stats, "smn", "child")
    smn_adolescent = queryStats(stats, "smn", "adolescent")
    smn_adult = queryStats(stats, "smn", "adult")
    smn_mixAge = queryStats(stats, "smn", "mix age")
    smn_noAge = queryStats(stats, "smn", "No Age")
    smn_acute = queryStats(stats, "smn", "Acute")
    smn_acuteSub = queryStats(stats, "smn", "ac/subac")
    smn_subacute = queryStats(stats, "smn", "subacute")
    smn_subChron = queryStats(stats, "smn", "subac/chron")
    smn_chronic = queryStats(stats, "smn", "chronic")
    smn_repSub = queryStats(stats, "smn", "repsub")
    smn_mixChron = queryStats(stats, "smn", "mix cnicity")
    smn_noChron = queryStats(stats, "smn", "No Cnicity")
    smn_sport = queryStats(stats, "smn", "sport")
    smn_military = queryStats(stats, "smn", "military")
    smn_civilian = queryStats(stats, "smn", "civilian")
    smn_mixType = queryStats(stats, "smn", "mix Type")
    smn_healthyControl = queryStats(stats, "smn", "HC")
    smn_noContactControl = queryStats(stats, "smn", "NCC")
    smn_inSportControl = queryStats(stats, "smn", "ISC")
    smn_otherControl = queryStats(stats, "smn", "Other Control")
    
    smn_categories = [smn, smn_mild, smn_mildMod, smn_mod, smn_modSev, smn_sev, 
                  smn_mixSev, smn_noSev, smn_child, smn_adolescent, smn_adult,
                 smn_mixAge, smn_noAge, smn_acute, smn_acuteSub, smn_subacute, smn_subChron, smn_chronic, smn_repSub, 
                 smn_mixChron, smn_noChron, smn_sport, smn_military, smn_civilian, smn_mixType, 
                 smn_healthyControl, smn_noContactControl, smn_inSportControl, smn_otherControl]
    
    smn_incList = []
    for cat in smn_categories:
        smn_incList.append(cat[0])
    smn_decList = []
    for cat in smn_categories:
        smn_decList.append(cat[1])
    smn_nullList = []

    for cat in smn_categories:
        smn_nullList.append(cat[2])
    
    data_smn = {'Increase': smn_incList, 
                'Decrease': smn_decList, 
                'Null': smn_nullList}        
    df_smn = pd.DataFrame(data_smn, index = detailed_index)
    display(df_smn)
    
    
#--end printStats function-----------------------------------------------------------

#RUN IT ON THE DATA!
printStats(stats)

Starting Counts...


Network Totals: 


,Increase,Decrease,Null
DMN,20,20,33
ECN,8,7,20
Limbic,3,2,6
Salience/VAN,3,4,11
DAN,1,0,6
Visual,3,3,8
Sensorimotor,4,3,9
Total,42,39,93


All Networks in Detail:


,Increase,Decrease,Null
All,42,39,93
Mild,27,30,53
Mild/Moderate,0,0,10
Moderate,0,0,0
Moderate/Severe,10,3,15
Severe,0,2,1
Mixed Severity,5,4,14
No Severity Stated,0,0,0
Child,0,0,0
Adolescent,1,2,4


Results by Severity & Chronicity


...

,Increase,Decrease,Null
All,1,0,6
Mild,0,0,6
Mild/Moderate,0,0,0
Moderate,0,0,0
Moderate/Severe,1,0,0
Severe,0,0,0
Mixed Severity,0,0,0
No Severity Stated,0,0,0
Child,0,0,0
Adolescent,0,0,0


Visual in Detail:


,Increase,Decrease,Null
All,3,3,8
Mild,3,3,4
Mild/Moderate,0,0,2
Moderate,0,0,0
Moderate/Severe,0,0,1
Severe,0,0,0
Mixed Severity,0,0,1
No Severity Stated,0,0,0
Child,0,0,0
Adolescent,0,0,0


SMN in Detail:


,Increase,Decrease,Null
All,4,3,9
Mild,3,3,5
Mild/Moderate,0,0,2
Moderate,0,0,0
Moderate/Severe,0,0,1
Severe,0,0,0
Mixed Severity,1,0,1
No Severity Stated,0,0,0
Child,0,0,0
Adolescent,0,0,0
